In [ ]:
# Here we implement the FCM algorithm
class FCM:
  """
  Simple Fuzzy C-Means clustering algorithm implementation.
    
  Parameters
  ----------
  n_clusters : int, default=2
      The number of clusters to form.
  fuzziness : float, default=2
      The fuzziness index, which must be greater than 1.
  max_iter : int, default=300
      The maximum number of iterations to run the algorithm.
  tol: float, default=1e-4
      Relative tolerance with regards to Frobenius norm of the difference
      in the cluster centers of two consecutive iterations to declare
      convergence.
  random_state : int, default=None
      Determines random number generation for centroid initialization. Use
      an int to make the randomness deterministic.
        
  Attributes
  ----------
  labels_ : ndarray of shape (n_samples,)
      The predicted labels of data points.
  n_iter_ : int
      Number of iterations run.
  cluster_centers_ : ndarray of shape (n_clusters, n_features)
      The coordinates of the final cluster centers.
  distance_: ndarray of shape (n_samples, n_clusters)
      The final squared distances between data points and cluster centers.
  membership_matrix_ : ndarray of shape (n_samples, n_clusters)
      The final fuzzy membership matrix.
  objective_ : float
      The membership weighted sum of squared distances of the samples to cluster centers.
  fpc_ : float
      The fuzzy partition coefficient.
    """

  def __init__(self, n_clusters=2, fuzziness=2, max_iter=300, tol=1e-4, random_state=None):
    self.n_clusters = n_clusters
    self.fuzziness = fuzziness
    self.max_iter = max_iter
    self.tol = tol
    self.random_state = random_state
    
  def fit(self, X):
    # Initialize cenrtoids randomly (we choose randomly n_clusters data points from our dataset)
    # We can initialize the centroids with numerous other ways (e.g. the initialization in 'k-means++' algorithm)
    # We can also random initialize the fuzzy membership matrix instead of the centroids with numerous ways (fixme: implement this and compare the results)
    # For better results we can run the algorithm multiple times with different random initializations and choose the one with the best results (fixme: implement this and compare the results)
    # Changes for time efficiency can also be done
    rng = np.random.default_rng(self.random_state)
    self.cluster_centers_ = X[rng.permutation(X.shape[0])[:self.n_clusters]]

    self.membership_matrix_ = np.ones((X.shape[0], self.n_clusters)) / self.n_clusters

    self.distance_ = np.zeros((X.shape[0], self.n_clusters))

    # Iterate until convergence or maximum number of iterations is reached   
    for i in range(1, self.max_iter+1):
      # Calculate squared Euclidean distance between data points and cluster centers (fixme: we can use other norms as well and compare the results)
      self.distance_ = self._compute_distance_matrix(X)

      # Update the fuzzy membership matrix
      self.membership_matrix_ = self._compute_membership_matrix()

      # Classify data points to the cluster with the maximum membership value
      self.labels_ = np.argmax(self.membership_matrix_, axis=1)

      # Update the centroids based on the current membership values
      new_centroids = self._compute_centroids(X)

      # Count the iterations of the algorithm so far
      self.n_iter_ =  i

      # Calculate the objective function
      self.objective_ = np.sum((self.membership_matrix_ ** self.fuzziness) * self.distance_)

      # Calculate the fuzzy partition coefficient (FPC)
      self.fpc_ =  np.sum(self.membership_matrix_ ** 2 / (X.shape[0]))
            
      # Check for convergence (with regards to Frobenius norm - we can use numerous other ways)
      error = np.linalg.norm(self.cluster_centers_ - new_centroids)
      if error < self.tol:
        break
      if i != self.max_iter: # do not update the centroids in last iteration
        self.cluster_centers_ = new_centroids # update the centroids if not converged yet
    
  def predict(self, X):
    # Assign data points to the cluster with the maximum membership value

    # Compute squared distance between data points and the trained model's centroids
    self.distance_ = self._compute_distance_matrix(X)

    # Compute the fuzzy membership matrix using the trained model's centroids
    self.membership_matrix_ = self._compute_membership_matrix()
    
    labels = np.argmax(self.membership_matrix_, axis=1)
    return labels
    
  def fit_predict(self, X):
    self.fit(X)
    return self.predict(X)

  def transform(self, X):
    # Transform data points to distance and membership matrices
    self.distance_ = self._compute_distance_matrix(X)

    return np.sqrt(self.distance_), self._compute_membership_matrix()

  def fit_transform(self, X):
    self.fit(X)
    return self.transform(X)

  def score(self, X):
    # Compute the negative objective function
    self.distance_ = self._compute_distance_matrix(X)

    # Compute the membership matrix using the trained model's centroids
    self.membership_matrix_ = self._compute_membership_matrix()

    return -np.sum((self.membership_matrix_ ** self.fuzziness) * self.distance_)
  
  def _compute_centroids(self, X):
    # Compute centroids using the current membership matrix
    return (X.T @ (self.membership_matrix_ ** self.fuzziness)).T / (self.membership_matrix_ ** self.fuzziness).sum(axis=0, keepdims=True)

  def _compute_membership_matrix(self):
    # Compute new membership matrix using the current centroids
    membership = 1 / (self.distance_**(1 / (self.fuzziness - 1)))
    membership = membership / membership.sum(axis=1, keepdims=True)
    return membership

  def _compute_distance_matrix(self, X):
    # Calculate squared Euclidean distance between data points and cluster centers (we can use other norms as well)
    distance = np.zeros((X.shape[0], self.n_clusters))
    for j in range(self.n_clusters):
      distance[:, j] = np.sum((X - self.cluster_centers_[j]) ** 2, axis=1)
    return distance + 1e-10 # we add a small constant so we don't divide by zero in membership values calculation